In [6]:
import pandas as pd
import string
import re
import nltk
import numpy as np
import collections as co
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [7]:
data = pd.read_csv('data/lenta-ru-news.csv')

In [8]:
data.head()

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество


In [9]:
data.shape

(739351, 5)

In [10]:
freq = pd.DataFrame(data['tags'].value_counts())

category_to_drop = ['Украина', 'Происшествия', 'Госэкономика', 'Следствие и суд', 'Преступность',
                    'Регионы', 'Криминал', 'Конфликты', 'Полиция и спецслужбы', 'Молдавия', 'Все',
                    'Общество', 'Политика', 'Прибалтика', 'Закавказье', 'Средняя Азия', 'Оружие',
                    'Космос', 'Белоруссия', 'Деловой климат', 'Реклама', 'Мнения', 'Преступная Россия',
                    'Киберпреступность', 'Вирусные ролики', 'Мемы', 'Мировой бизнес', 'Автобизнес',
                    'Финансы компаний', 'Явления', 'Пресса'
                   ]

freq_threshold = 100
category_to_drop.extend(freq[freq['tags'] < freq_threshold].index)

In [11]:
for cat in category_to_drop:
    data = data.drop(data[data['tags'] == cat].index)

In [12]:
data.shape

(135384, 5)

In [13]:
data['tags'].unique()

array(['Зимние виды', 'Мир', 'Coцсети', 'Хоккей', 'Деньги', 'Футбол',
       'Бизнес', 'Музыка', 'Квартира', 'Бокс и ММА', 'Театр', 'Дача',
       'Рынки', 'Звери', 'Техника', 'Интернет', 'Люди', 'Наука',
       'Внешний вид', 'Кино', 'ТВ и радио', 'Стиль', 'События',
       'Искусство', 'Движение', 'Жизнь', 'Авто', 'Книги', nan, 'Часы',
       'Летние виды', 'Россия', 'Вещи', 'История', 'Игры', 'Достижения',
       'Офис', 'Еда', 'Москва', 'Гаджеты', 'Город', 'Софт', 'Инструменты'],
      dtype=object)

In [14]:
data['tags'].isna().value_counts()

False    108870
True      26514
Name: tags, dtype: int64

In [15]:
data = data.drop(data[data['tags'].isna()].index)

In [16]:
data.shape

(108870, 5)

In [17]:
data['title'] = data['title'].str.replace('\xa0', ' ')
data['title'] = data['title'].str.replace('\[.*\]', '', regex=True)
data['title'] = data['title'].str.replace("\!", '', regex=True)
data['title'] = data['title'].str.replace("\'", '', regex=True)
data['title'] = data['title'].str.lower()

In [18]:
df_reviews = data[['title']][:]
df_reviews.columns = ['review']
df_reviews.reset_index(drop = True , inplace = True)
df_reviews.head(5)

,review
0,австрия не представила доказательств вины росс...
1,обнаружено самое счастливое место на планете
2,российскую молодежь предложили обучать духовны...
3,овечкин повторил свой рекорд
4,названы регионы россии с самым дорогим и дешев...


In [19]:
list_true_words = []

for i ,row in  df_reviews.iterrows():
    words = []    
    true_words = []    
    
    a_line = row['review']
    
    try:
        words = re.split(' ', a_line)
        true_words = []

        for word in words:
            m = re.search('(\w+)',word)
            if m is not None:
                good_word = m.group(0)
                true_words.append(good_word)       
    except:
        pass
    
    list_true_words.append(true_words)

In [20]:
df_reviews['list_of_words'] = list_true_words

In [21]:
text_analizator_df = df_reviews[:]
text_analizator_df.rename(columns = {'review' : 'sentence'} , inplace = True)
text_analizator_df.head()

,sentence,list_of_words
0,австрия не представила доказательств вины росс...,"[австрия, не, представила, доказательств, вины..."
1,обнаружено самое счастливое место на планете,"[обнаружено, самое, счастливое, место, на, пла..."
2,российскую молодежь предложили обучать духовны...,"[российскую, молодежь, предложили, обучать, ду..."
3,овечкин повторил свой рекорд,"[овечкин, повторил, свой, рекорд]"
4,названы регионы россии с самым дорогим и дешев...,"[названы, регионы, россии, с, самым, дорогим, ..."


In [22]:
# Лемматизация
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize_list_of_words(list_of_words):
    res = []
    for word in list_of_words:
        p = morph.parse(word)[0]
        new_word = p.normal_form
        res.append(new_word)
    return res  

tqdm.pandas(desc="my bar!")
text_analizator_df['lemmatize_list_of_words'] = text_analizator_df['list_of_words'].progress_apply(lemmatize_list_of_words)

c:\python\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
my bar!: 100%|████████████████████████████████████████████████████████████████| 108870/108870 [01:50<00:00, 985.18it/s]


In [23]:
text_analizator_df.head()

,sentence,list_of_words,lemmatize_list_of_words
0,австрия не представила доказательств вины росс...,"[австрия, не, представила, доказательств, вины...","[австрия, не, представить, доказательство, вин..."
1,обнаружено самое счастливое место на планете,"[обнаружено, самое, счастливое, место, на, пла...","[обнаружить, самый, счастливый, место, на, пла..."
2,российскую молодежь предложили обучать духовны...,"[российскую, молодежь, предложили, обучать, ду...","[российский, молодёжь, предложить, обучать, ду..."
3,овечкин повторил свой рекорд,"[овечкин, повторил, свой, рекорд]","[овечкин, повторить, свой, рекорд]"
4,названы регионы россии с самым дорогим и дешев...,"[названы, регионы, россии, с, самым, дорогим, ...","[назвать, регион, россия, с, самый, дорогой, и..."


In [24]:
f = lambda x: ' '.join([item for item in x])
text_analizator_df['lem_text'] = text_analizator_df['lemmatize_list_of_words'].apply(f)

In [25]:
print(f'old- {text_analizator_df.loc[0]["sentence"]}' )
print('\n')
print(f'stem -{text_analizator_df.loc[0]["lem_text"]}' )

old- австрия не представила доказательств вины российских биатлонистов


stem -австрия не представить доказательство вина российский биатлонист


In [26]:
df_train = pd.DataFrame({'txt': text_analizator_df['lem_text'], 'target': data['tags'].reset_index(drop=True)})

In [27]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train['txt'],
                                                    df_train['target'], 
                                                    test_size=0.2,
                                                    stratify = df_train['target'] 
                                                   )

In [28]:
df_fasttext_train = pd.DataFrame({'txt': x_train, 'target': y_train})
df_fasttext_test = pd.DataFrame({'txt': x_test, 'target': y_test})

In [29]:
# Для обучения придется сделать файл, где целевой класс должен начинаться с __label__
df_fasttext_train['target'] = df_fasttext_train['target'].apply(lambda x: '__label__' + str(x))
df_fasttext_test['target'] = df_fasttext_test['target'].apply(lambda x: '__label__' + str(x))
df_fasttext_train[['target', 'txt']].to_csv('df_fasttext_train.txt', header=False, index=False, sep="\t")
df_fasttext_test[['target', 'txt']].to_csv('df_fasttext_test.txt', header=False, index=False, sep="\t")

In [30]:
df_train['target'] = df_train['target'].apply(lambda x: '__label__' + str(x))
df_train[['target', 'txt']].to_csv('df_fasttext_train_fulldata.txt', header=False, index=False, sep="\t")

In [50]:
import fasttext
model = fasttext.train_supervised(input='df_fasttext_train_fulldata.txt',
                                  dim=80,
                                  epoch=50,
                                  lr=0.5,
                                  wordNgrams=3,
                                  ws=10                                  
                                 )

In [51]:
model.test("df_fasttext_test.txt", k=1)

(21774, 0.9994948103242399, 0.9994948103242399)

In [52]:
model.save_model('models/fasttext_fulldata.bin')

In [53]:
def get_tag(prediction):
    tag = str(prediction[0])
    proba = prediction[1][0]
    tag = tag.replace('__label__', '')
    tag = list(filter(None, re.split('\W|\d', tag)))
    return tag[0] if (proba > 0.9) else -1

In [54]:
message = 'купить акция и облигация'
prediction = model.predict(message, k=1)
prediction

(('__label__Рынки',), array([0.9757992]))

In [55]:
get_tag(prediction)

'Рынки'

In [56]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [1]:
# with the previously trained `model` object, call :
model.quantize(input='df_fasttext_train_fulldata.txt', retrain=True)

# then display results and save the new model :
print_results(*model.test("df_fasttext_test.txt"))
model.save_model("models/fasttext_fulldata_2.ftz")

NameError: name 'model' is not defined

In [58]:
help(model.quantize)

Help on method quantize in module fasttext.FastText:

quantize(input=None, qout=False, cutoff=0, retrain=False, epoch=None, lr=None, thread=None, verbose=None, dsub=2, qnorm=False) method of fasttext.FastText._FastText instance
    Quantize the model reducing the size of the model and
    it's memory footprint.

